# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
subject_id = 20902040

In [4]:
# Parameters
subject_id = 20902029


In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/30 [00:00<?, ?it/s]

Rendering models:   7%|▋         | 2/30 [00:06<01:37,  3.49s/it]

Rendering models:  10%|█         | 3/30 [00:07<01:08,  2.54s/it]

Rendering models:  17%|█▋        | 5/30 [00:07<00:45,  1.81s/it]

Rendering models:  20%|██        | 6/30 [00:07<00:33,  1.39s/it]

...

Rendering models:  90%|█████████ | 27/30 [00:14<00:00,  3.71it/s]

Rendering models:  93%|█████████▎| 28/30 [00:14<00:00,  4.05it/s]

Rendering models:  97%|█████████▋| 29/30 [00:15<00:00,  3.43it/s]

Rendering models: 100%|██████████| 30/30 [00:15<00:00,  3.79it/s]

Sammy_Lawrence11                       0.001605
not-logged-in-545b59d533d78a4f8dea     0.001333
not-logged-in-8c0c44faf754d344b18a     0.040906
equidad1                               0.155173
not-logged-in-aac1a549b28981db19c0     0.000929
CTidwell3                              0.000958
Betelgeuse12                           0.000823
not-logged-in-7fda7748d862237450e1     0.009849
not-logged-in-587a4e9ad373b8ff5983     0.001540
lsautter                               0.000710
not-logged-in-6ead6661c7dcef165774     0.004632
coyotes54                              0.000588
Raven_Vasquez                          0.002472
orzechowskijacek                       0.111981
BirdLeaf                               0.000567
not-logged-in-1df2dc33157510bbec3e    29.243422
clairedeu                              0.000747
Dr.Death                               0.006187
Phoebe14                               0.047185
acapirala                              0.001265
Lavadude                               0

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())